This notebook will produce all the model results shown in:
Numerical modeling of groundwater-driven stream network evolution in low relief areas
by Cecilia Cullen, Alison M. Anders, Jingtao Lai, and Jennifer L. Druhan,
submitted to Earth Surface Processes and Landforms 1/10/21 
in revision 5/3/21



In [ ]:
# Importing components from Landlab and setting up the model grid
import numpy as np
from pylab import *
from landlab import RasterModelGrid
from landlab.components.flow_accum import FlowAccumulator
from landlab.components.stream_power import FastscapeEroder
from landlab.field.scalar_data_fields import FieldError
from landlab.plot import channel_profile as prf
from landlab.components import LinearDiffuser
from landlab.plot import imshow_grid
from landlab.components import DepressionFinderAndRouter
from landlab.components import FlowDirectorD8
from landlab.io.esri_ascii import read_esri_ascii
from landlab.io.esri_ascii import write_esri_ascii

#setting up grid size

row_col = 100 #number of rows and cols
cell_size = 50 #m
mg = RasterModelGrid((row_col, row_col), cell_size)
M_row_col = mg.number_of_nodes

#setting up boundary conditions
mg.set_closed_boundaries_at_grid_edges(False, True, False, True)
mg.set_fixed_value_boundaries_at_grid_edges(False, False, True, False)


We used a model initial condition that included an incised valley along the left edge of the domian and a flat plateau of 15 m elevation with a small amount of random noise. 
The cell below was used to produce the initial topography.  
We used a single initial topography for all the models presented.  That topography is available in this archive as Seed50.txt

In [ ]:
#setting up grid size and resolution

row_col = 100 #number of rows and cols
M_row_col = row_col * row_col
cell_size = 10 #m
mg = RasterModelGrid((row_col, row_col), cell_size)

#initial topography is 15 m elevation except along left edge

z = np.ones(M_row_col, dtype=float) * 15 # m
z_step = z.reshape(row_col,row_col)
z_step[:,0] = 0
z = z_step.reshape(M_row_col)
z += np.random.rand(len(z))/1 #this adds noise to the model of up to 1 m

mg.add_field('node','topographic__elevation', z ,units = 'm')
savepath = './' #means current folder
from landlab.io.esri_ascii import write_esri_ascii
write_esri_ascii(savepath +'new_initial_topography.txt', mg, 'topographic__elevation') #note 

In [2]:
# Reading in the same initial random seed topography for all parameter sets
# Instead of creating a new initial topography, use this line to read in the initial topography we used for all models presented
(mg, z) = read_esri_ascii("Seed50.txt", name = 'topographic__elevation')

Two sets of models were run - one set with flow routing out of closed depressions so that all precipitation falling on the domain was forced to the model edge and another with no routing of surface water from closed depressions - the two sets of set-ups are shown below

In [3]:
# Instantiating landlab components for simulations with routing out of closed depressions.
ld = LinearDiffuser(mg, linear_diffusivity=0.001) #diffusivity fixed for all simulations presented
df = DepressionFinderAndRouter(mg)
fr = FlowAccumulator(mg, flow_director = FlowDirectorD8, depression_finder = DepressionFinderAndRouter)
sp = FastscapeEroder(mg, K_sp=0.0001, m_sp=0.5, threshold_sp = 0.0005, discharge_name = 'surface_water__discharge') 
# K_sp, m_sp, threshold_sp all fixed for all simulations presented


In [ ]:
#instantiation of Landlab components for simulations without routing out of closed depressions
ld = LinearDiffuser(mg, linear_diffusivity=0.001) #diffusivity fixed for all simulations presented
fr = FlowAccumulator(mg, flow_director = FlowDirectorD8)
sp = FastscapeEroder(mg, K_sp=0.0001, m_sp=0.5, threshold_sp = 0.0005, discharge_name = 'surface_water__discharge')
# K_sp, m_sp, threshold_sp all fixed for all simulations presented

Next we set up the amount of precipitation and the amount of groundwater.
In simulations presented we varied the fraction of precipitation vs. groundwater, but kept the total water added to the domain constant at 25e6 m3/yr, which is equivalennt to 1m/yr water depth added across the 25e6 m2 area of the domain.

In [4]:
#select the values for precipitation and groundwater
total_rain_out = 2.5e6 # cubic meters/yr across the entire domain 
#simulations presented include values of: 25.0e6,22.5e6,20.0e6,17.5e6,15.0e6,12.5e6,10.0e6,7.5e6,5.0e6,2.5e6
rain_rate = total_rain_out/(mg.dx*mg.dy*mg.number_of_core_nodes)
total_gw_in = 22.5e6#cubic meters/yr added to right side boundary
#simulations presented include values of: 0, 2.5e6, 5.0e6, 7.5e6, 10.0e6, 12.5e6,15.0e6, 17.5e6, 20.0e6, 22.5e6
percent_groundwater = total_gw_in/(total_rain_out + total_gw_in ) * 100

In [5]:
# some setting up of the groundwater model
#constants fed to the SeepCharge
k_gw = 1000 #m/yr
S = 0.0001
b =20 #thickness of aquifer
del_t=1
Qr = total_gw_in/(row_col-2) # this is spreading the flux of groundwater across the entire right boundary
Qt = 0 #flux into top boundary
Qb = 0 #flux into bottom boundary
L = cell_size
A = L * L
tolerance = 10e-7
time = 0

#setting up the extra fields in the groundwater model - the head, groundwater flux 

head = np.ones(mg.number_of_nodes)
depth = 100

#the initial condition for the hydraulic head is a linear decrease from the right boundary to the left edge which is fixed at head = 0
initial_high_head = (Qr* row_col)/(depth* k_gw) 
initial_slope = initial_high_head/(row_col)
head = head.reshape(row_col,row_col)
for i in range(0, (row_col)):
    head[:,i]=initial_slope*(i)
    
#adding grids to the model to store values we will compute    
#Field to store hydraulic head values    
head = head.reshape(mg.number_of_nodes,1)
gwhead = mg.add_field('node','hydraulic_head', head, units = 'm', copy = False, noclobber = False)

#Field to store the unit groundwater flux
groundwater_flux= np.ones(mg.number_of_nodes)
gwflux = mg.add_field('node','unit_groundwater_flux', groundwater_flux, units = 'm', copy = False, noclobber = False)
groundwater_flux.fill(0)

#Field to store whether a that part of the landscape will have seeps
isseep = np.ones(mg.number_of_nodes, dtype = int)
seepnodes = mg.add_field('node','seep_node', isseep, copy = False, noclobber = False)
isseep.fill(0)

#create dummy field that will store groundwater flux                
dummy_flux= np.ones(mg.number_of_nodes)
dummy = mg.add_field('node','dummy_flux', dummy_flux, units = 'm3/yr', copy = False, noclobber = False)
groundwater_flux.fill(0)

#Field to store surface water values
surfacewater = np.ones(mg.number_of_nodes)
surfacewater.fill(rain_rate)
wtradd = mg.add_field('node','water__unit_flux_in', surfacewater, units = 'm', copy = False, noclobber = False)

In [6]:
#Groundwater Model
"""Someday this will be revised into a more general component and added to the landlab system.  
For the simulations presented we used this as a class defined within a python script that included all the pieces in this 
notebook. """

class Underground_Flow_Routing:
    
    """   
    Calculate steady-state groundwater head distribution and groundwater
    flux under constant head or constant flux boundary conditions
    """

    #these are all the imported values from a different script
    def __init__(self, mg, k_gw , S, b , del_t, time, tolerance, high_head):  
        self.mg = mg
        self.k_gw = k_gw
        self.S = S
        self.b = b
        self.del_t = del_t
        self.time = time
        self.tolerance = tolerance
        self.high_head = high_head


    """ This piece identifies locations eroded below the chosen depth to the confined aquifer - 
    we use a fixed elevation of 10 m for all simulations presented in the paper """    
    def findseeps(self):
        
        topo = self.mg.at_node['topographic__elevation']
        topo  = topo.reshape(row_col, row_col)
        
        head = self.mg.at_node['hydraulic_head']
        head  = head.reshape(row_col, row_col)
        
        isseep = self.mg.at_node['seep_node']
        isseep  = isseep.reshape(row_col, row_col)
        #for i in range(0,mg.number_of_nodes):
        
        for i in range(0,(row_col)):
            for j in range(0,(row_col-1)):  
                if topo[i,j] < 10: #m
                        head[i,j] = 0
                        isseep[i,j]=1
                        
                isseep[0,0]= 0
                isseep[(row_col-1),0]= 0
        

        return mg

    """ This is the heart of the model - it takes the boundary conditions and the locations of the seeps and
    iteratively solves for the steady-state head distribution consistent with the specified groundwater flux into
    the model domain and locations of the seeps"""
                    
    def HeadCalc(self):
        
        H_old = np.ones(mg.number_of_nodes)
        depth = 100
        initial_high_head = (Qr* row_col)/(depth* k_gw) 
        initial_slope = initial_high_head/(row_col)
        H_old = H_old.reshape(row_col,row_col)
        for i in range(0, (row_col)):
            H_old[:,i]=initial_slope*(i)
        
        
        isseep = self.mg.at_node['seep_node']
        isseep  = isseep.reshape(row_col, row_col)
        
        
        H_new = zeros(mg.number_of_nodes)
        

        for i in range(0,row_col):
            for j in range(0,row_col):
                if isseep[i,j] == 1:
                        #head[i,j] = 0
                        H_old[i,j] = 0
        maxdiff = []
        howlong = []
        
        
        H_old  = head.reshape(mg.number_of_nodes)
        
        
        difference = H_new - H_old 
        
        biggest_diff = max(abs(difference))

        while biggest_diff > tolerance:
            
            H_new = H_new.reshape(row_col,row_col)
            H_old  = H_old.reshape(row_col, row_col)
            
            #Right Boundary 
            for k in range(0,row_col):
                H_new[k,(row_col-1)] = (Qr/(k_gw * mg.dx)) + H_old[k,(row_col-2)] 
                        
            #Left Boundary
            for j in range(0,row_col):
                H_new[j,0] = 0
                
            #Top Boundary
            for i in range(0,row_col):
                H_new[0,i] = H_old[1,i]+ (Qt/(k_gw * mg.dx))   
                
            #Bottom Boundary
            for i in range(0,row_col):
                H_new[(row_col-1),i] = H_old[(row_col-2),i]+ (Qb/(k_gw * mg.dx))   

                    
            for i in range(1, (row_col-1)):
                for j in range(1, (row_col - 1)):
                    #seeps 
                    if isseep[i,j] == 1:
                        H_new[i,j] = 0    
    
                    else:
                        H_new[i,j] = (H_old[i+1,j] *k_gw + H_old[i, j+1]*k_gw + H_old[i-1,j]*k_gw + H_old[i,j-1]*k_gw)/(k_gw*4)
            
            
            H_old  = H_old.reshape(mg.number_of_nodes)
            
            
            H_new = H_new.reshape(mg.number_of_nodes)
            
            biggest_diff = max(abs(H_new - H_old))
            
            maxdiff.append(biggest_diff)
            
            H_old = H_new
            H_new = zeros(mg.number_of_nodes)
            
            
        self.mg.at_node['hydraulic_head'] = H_old
    
           
        return mg
    
    """ This piece computes the groundwater flux to seeps using the steady-state groundwater head distribution
    calculated above. The flux of groundwater to the seeps will be added to the surface water within the time loop in landlab"""    
    def SeepCharge(self):
        b = 20
        
        head_gradient = self.mg.calc_grad_at_link('hydraulic_head')
        fluxonlinks = k_gw* abs(head_gradient)
        isseep = self.mg.at_node['seep_node']
        dummy_flux = self.mg.at_node['dummy_flux'] 
        seepflux = np.zeros(mg.number_of_nodes)
        
        for i in range(0,mg.number_of_nodes):
            if isseep[i] == 1:
                linkstosum=[]
                sharedlinks =[]
                templist = mg.adjacent_nodes_at_node[i]
                templinks = mg.links_at_node[i]
                
                for j in range(0,4):
                    if isseep[templist[j]] != 1:
                        if templist[j]!= -1:
                            sharedlinks.append(templinks[j])

                for x in range(0,len(sharedlinks)):
                    if sharedlinks[x] != -1:
                        seepflux[i]+=fluxonlinks[sharedlinks[x]]
        self.mg.at_node['dummy_flux'] = seepflux

        return mg

In [7]:
#instantiation of the class defined above.
gw = Underground_Flow_Routing(mg, k_gw, S, b, del_t, time, tolerance, initial_high_head)

Here we need to have two versions - one with flow routing out of closed depressions and one without.  Those two versions are given in the two cells that follow

In [ ]:
# Start of time loop for flow routing cases
dt = 1000 #yrs, timestep for stream power erosion
number_time_steps = 60 #60,000 year simulations
uplift_rate = 0 #modeling response to glacial meltwater carved valley without uplift

total_water_out  = []
total_seep_flux = []
Precip_added = []


for o in range(number_time_steps):
    #reset surface water
    surfacewater.fill(rain_rate)

    #call groundwater to find the nodes that are seeps
    gw.findseeps()

    #update the hydraulic head distribution
    gw.HeadCalc()

    # find the flux of groundwater at seeps
    gw.SeepCharge()

    #add groundwater flux to seeps to to surface water
    temp = mg.at_node['dummy_flux']
    total_sf = sum(temp)* mg.dx * mg.dy
    total_seep_flux.append(total_sf)
    surfacewater += temp#/(cell_size*cell_size)
    
    #reset temp to 0
    temp.fill(0)
    
    #surface water is routed downslope
    fr.run_one_step()

    #depression finder and router
    df = DepressionFinderAndRouter(mg)
    df.map_depressions()

    #stream-power based erosion
    sp.run_one_step(dt=1000)
    
    #linear hillslope diffusion
    ld.run_one_step(dt)
    
    #write output to files for post processing
    if o%5 == 0:
        write_esri_ascii('topo_90per_{}.txt'.format(o), mg, 'topographic__elevation')
        write_esri_ascii('swd__90per__{}.txt'.format(o), mg, 'surface_water__discharge')
        write_esri_ascii('da__90per__{}.txt'.format(o), mg, 'drainage_area')


In [ ]:
# Start of time loop for  cases with no flow routing
dt = 1000 #yrs, timestep for stream power erosion
number_time_steps = 60 
uplift_rate = 0


total_water_out  = []
total_seep_flux = []
Precip_added = []


for o in range(number_time_steps):
    #reset surface water
    surfacewater.fill(rain_rate)

    #call groundwater to find the nodes that are seeps
    gw.findseeps()

    #update the hydraulic head distribution
    gw.HeadCalc()

    # find the flux of groundwater to seeps
    gw.SeepCharge()

    #add groundwater flux to seeps to to surface water
    temp = mg.at_node['dummy_flux']
    total_sf = sum(temp)* mg.dx * mg.dy
    total_seep_flux.append(total_sf)
    surfacewater += temp#/(cell_size*cell_size)
    
    #reset temp to 0
    temp.fill(0)
    
    #surface water is routed downslope
    fr.run_one_step()

    #stream-power based erosion
    sp.run_one_step(dt=1000)
    
    #linear hillslope diffusion
    ld.run_one_step(dt)
    
    #write output to files for post processing
    if o%5 == 0:
        write_esri_ascii('topo_NR_STD_0per_{}.txt'.format(o), mg, 'topographic__elevation')
        write_esri_ascii('swd__NR_STD_0per__{}.txt'.format(o), mg, 'surface_water__discharge')
        write_esri_ascii('da__NR_STD_0per__{}.txt'.format(o), mg, 'drainage_area')
